In [1]:
a = 0
b = 30

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pickle
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)
ks11 = pd.read_csv('ks11.csv')
ks11.index = ks11.Date
del ks11['Date']

copy_keys = list(raw_data.keys())
for stock in copy_keys:
    if len(raw_data[stock].loc[:'2013-01-01']) < 1:
        del raw_data[stock]
        
ks11 = ks11.drop(ks11.index[np.where(ks11.Volume==0)[0]])
        
for key, df in raw_data.items():
    raw_data[key] = df.drop(df.index[np.where(df.Volume==0)[0]], axis=0)
    
# data = ks11.copy()
# data = raw_data['삼성전자'].copy()
# data = raw_data['NAVER'].copy()
# data = raw_data['LG화학'].copy()
data = raw_data['미래에셋대우'].copy()

data["pct_change"] = data.Close.pct_change()
for i in range(3, 31, 2):
    data["close_ma_{}".format(i)] = data.Close.rolling(window = i).mean()
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    data["his_vol_{}".format(i)] = data["pct_change"].rolling(window = i).std()*(252**0.5)
for i in [1, 3, 5, 7, 10, 12, 14]:
    data['returnsCC{}'.format(i) ]= np.log(data.Open / data.Close.shift(i))
    data['returnsOO{}'.format(i) ] = np.log(data.Open / data.Open.shift(i))
    data['returnsCC{}'.format(i) ]= np.log(data.Close / data.Close.shift(i))
    data['returnsOC{}'.format(i) ] = np.log(data.Close / data.Open.shift(i))
for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
          55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
    for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        data['Y1_{}_s{}'.format(i, shift)] = (data.Close - data.Close.shift(shift).rolling(i).mean())
        data['Y3_{}_s{}'.format(i, shift)] = (data.Close.rolling(i).mean() - data.Close.shift(shift).rolling(i).mean())
        data['sh{}_Y1_{}'.format(shift, i)] = np.where((data['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        data['sh{}_Y3_{}'.format(shift, i)] = np.where((data['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        
data_ = data.loc['2018-07-01':'2018-10-01']
data = data.loc['2013-07-01':'2018-07-01']

X_li = [i for i in data.columns if (i.find('sh') == -1) if (i.find('Volume') == -1)]

X_li2 = [i for i in data.columns 
         if (i.find('sh') == -1) 
         if (i.find('Volume') == -1)
         if (i.find('Y') == -1)]

%run moon_code
%run xgb_modelfit

import copy

train, test = train_test_split(data, 0.8, dtrain=True)

y_li = [i for i in data.columns if i.find('sh') != -1]

def modelfit(alg, train, test, X, y):
    alg = copy.deepcopy(alg)
    xgb_params = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(train[X].values, label=train[y].values)
    cvresult=xgb.cv(xgb_params, xgtrain,
                    num_boost_round=alg.get_params()['n_estimators'],
                    nfold=5, early_stopping_rounds=50,
                    show_stdv=False)
    alg.set_params(n_estimators=cvresult.shape[0])
    alg.fit(train[X], train[y], eval_metric='auc')
    dtrain_predictions = alg.predict(train[X])
    dtrain_predprob = alg.predict_proba(train[X])[:,1]
    dtest_predictions = alg.predict(test[X])
    dtest_predprob = alg.predict_proba(test[X])[:,1]
#     print("Train Accuracy : {:.4f}".format(metrics.accuracy_score(
#                             train[y].values, dtrain_predictions)))
#     print("Train AUC Score : {:.4f}".format(metrics.roc_auc_score(
#                             train[y].values, dtrain_predprob)))
#     print("Test  Accuracy : {:.4f}".format(metrics.accuracy_score(
#                             test[y].values, dtest_predictions)))
#     print("Test  AUC Score : {:.4f}".format(metrics.roc_auc_score(
#                             test[y].values, dtest_predprob)))
    return alg

for idx, stock in enumerate(raw_data.keys()):
        
    if idx < a : 
        continue
    if idx > b :
        break
        
    print(stock)
    data = raw_data[stock].copy()

    data["pct_change"] = data.Close.pct_change()
    for i in range(3, 31, 2):
        data["close_ma_{}".format(i)] = data.Close.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        data["his_vol_{}".format(i)] = data["pct_change"].rolling(window = i).std()*(252**0.5)
    for i in [1, 3, 5, 7, 10, 12, 14]:
        data['returnsCC{}'.format(i) ]= np.log(data.Open / data.Close.shift(i))
        data['returnsOO{}'.format(i) ] = np.log(data.Open / data.Open.shift(i))
        data['returnsCC{}'.format(i) ]= np.log(data.Close / data.Close.shift(i))
        data['returnsOC{}'.format(i) ] = np.log(data.Close / data.Open.shift(i))
    for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
              55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
        for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
            data['Y1_{}_s{}'.format(i, shift)] = (data.Close - data.Close.shift(shift).rolling(i).mean())
            data['Y3_{}_s{}'.format(i, shift)] = (data.Close.rolling(i).mean() - data.Close.shift(shift).rolling(i).mean())
            data['sh{}_Y1_{}'.format(shift, i)] = np.where((data['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
            data['sh{}_Y3_{}'.format(shift, i)] = np.where((data['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)

    data_ = data.loc['2018-07-01':'2018-10-01']
    data = data.loc['2013-07-01':'2018-07-01']

    X_li = [i for i in data.columns if (i.find('sh') == -1) if (i.find('Volume') == -1)]

    train, test = train_test_split(data, 0.8, dtrain=True)

    y_li = [i for i in data.columns if i.find('sh') != -1]

    model_set = {}
    for y in y_li:
        model = modelfit(alg, train, test, X_li, y)
        model_set[y] = model
        
    try:    
        y1 = pd.DataFrame()
        y3 = pd.DataFrame()
        for idx, y in enumerate(y_li):
            acc = metrics.accuracy_score(test[y], model_set[y].predict(test[X_li]))
            auc = metrics.roc_auc_score(test[y], model_set[y].predict(test[X_li]))
            res = pd.DataFrame([acc,auc], index=['accuracy', 'AUC'], columns=[y])
            if idx%2==0:
                y1 = pd.concat((y1, res), axis=1)
            else:
                y3 = pd.concat((y3, res), axis=1)

        Y1 = np.array([])
        for i in range(int(270/10)):
            i *= 10
            if i == 0:
                Y1 = y1.values[:,i:i+10]
            else:
                Y1 = np.vstack((Y1, y1.values[:,i:i+10]))
    except ValueError:
        continue

    Y3 = np.array([])
    for i in range(int(270/10)):
        i *= 10
        if i == 0:
            Y3 = y3.values[:,i:i+10]
        else:
            Y3 = np.vstack((Y3, y3.values[:,i:i+10]))

    Y = np.hstack((Y3, Y1))

    y1.to_excel('./20181105_result/y1_{}.xlsx'.format(stock))
    y3.to_excel('./20181105_result/y3_{}.xlsx'.format(stock))
    pd.DataFrame(Y).to_excel('./20181105_result/Y_{}.xlsx'.format(stock))

    with open('./20181105_result/model_set_{}_20181031.pickle'.format(stock), 'wb') as handle:
        pickle.dump(model_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

AK홀딩스


KeyboardInterrupt: 